# Using LLMs

This Jupyter Notebook demonstrates the use of various Large Language Model (LLM) providers and tools for natural language processing tasks. It showcases how to interact with LLMs using the `crewai` library (which is used by Agentics), parse structured outputs with Pydantic models, and explore available LLM connections. The notebook provides practical examples for calling different LLMs, formatting responses, and integrating with external providers such as OpenAI and IBM WatsonX.

In [1]:
! uv pip install agentics-py


import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive

    drive.mount("/content/drive")
    from google.colab import userdata

    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 64ms
In Colab: False


## Connect to your own LLM provider
Agentics uses CrewAI wrappers for main LLM providers. You can initialize your LLM as follows.
[find out more](https://docs.crewai.com/en/concepts/llms)

In [2]:
from crewai import LLM

# pick a provider (openai, anthropic, groq, etc.) - see crewai docs for details
llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.7,  # Adjust based on task
    max_tokens=4096,  # Set based on output needs
    timeout=300,
)  # Longer timeout for complex tasks

print(llm)

## Perform Simple LLM call

Once an LLM is instatiated, you can perform LLM calls

In [3]:
print(llm.call("where is the Eiffel Tower?"))

The Eiffel Tower is located in **Paris, France**.



## Perform Structured Call

LLMs can generate structured objects given a pydantic schema if you instantiate them accordingly

In [4]:
from pydantic import BaseModel


class Answer(BaseModel):
    short_answer: str
    detailed_answer: str
    confidence: float


struct_llm = LLM(model="gemini/gemini-2.0-flash", response_format=Answer)

print(struct_llm.call("Why is the sky blue?"))

{
  "short_answer": "The sky is blue because of a phenomenon called Rayleigh scattering.",
  "detailed_answer": "Sunlight is made up of all the colors of the rainbow. When sunlight enters the Earth's atmosphere, it collides with tiny air molecules. This causes the light to scatter in different directions. Blue and violet light have shorter wavelengths, and are scattered more than other colors, such as red and orange. Because blue light is scattered the most, it reaches our eyes from all directions, making the sky appear blue. Violet light is scattered even more, but our eyes are less sensitive to violet, and the sun emits less violet light, so the sky appears blue instead of violet. At sunrise and sunset, the sunlight has to travel through more of the atmosphere to reach us. This means that more of the blue light is scattered away, leaving the red and orange light to dominate, which is why sunsets are often red and orange.",
  "confidence": 0.95
}


## You can also use structured decoding for information extraction

In [5]:
import requests


class Person(BaseModel):
    name: str
    city: str
    occupation: str


class Book(BaseModel):
    title: str
    publisher: str
    year: int
    author: Person


class InformationExtraction(BaseModel):
    books: list[Book]
    people: list[Person]


person_llm = LLM(model="gemini/gemini-2.0-flash", response_format=InformationExtraction)
print(person_llm.call(requests.get("https://iep.utm.edu/wittgens/").text))

{
  "books": [
    {
      "title": "Tractatus Logico-Philosophicus",
      "publisher": "Routledge and Kegan Paul",
      "year": 1961,
      "author": {
        "name": "Ludwig Wittgenstein",
        "city": "Vienna",
        "occupation": "Philosopher"
      }
    },
    {
      "title": "Philosophical Investigations",
      "publisher": "Basil Blackwell",
      "year": 1963,
      "author": {
        "name": "Ludwig Wittgenstein",
        "city": "Vienna",
        "occupation": "Philosopher"
      }
    },
    {
      "title": "Wittgenstein on Rules and Private Language",
      "publisher": "Harvard University Press",
      "year": 1982,
      "author": {
        "name": "Saul A. Kripke",
        "city": "New York",
        "occupation": "Philosopher"
      }
    }
  ],
  "people": [
    {
      "name": "Ludwig Wittgenstein",
      "city": "Vienna",
      "occupation": "Philosopher"
    },
    {
      "name": "Bertrand Russell",
      "city": "Cambridge",
      "occupation": "Philo

## Using Agentics Predefined LLMs

Agentics has the following LLM handles: watsonx_llm, openai_llm, gemini_llm
You can directly import and use them as defaults

In [6]:
from agentics import AG

print(AG.get_llm_provider("list"))

llm = AG.get_llm_provider()  ## get the default LLM provider from the available ones
print(llm.call("Where is Rome?"))

llm = AG.get_llm_provider(
    "gemini"
)  ## get a specific LLM provider from the available ones,change "gemini" with "openai", "watsonx", etc. as needed
print(llm.call("Where is Rome?"))

{'watsonx': <crewai.llm.LLM object at 0x14d4f3800>, 'gemini': <crewai.llm.LLM object at 0x14d24b890>, 'openai': <crewai.llm.LLM object at 0x14d4683b0>}
Rome is the capital city of Italy. It lies in the central‑western part of the Italian Peninsula, on the banks of the Tiber River. Geographically, its coordinates are approximately **41.90° N latitude and 12.48° E longitude**. The city is situated in the region of **Lazio**, about 24 km (15 mi) inland from the Tyrrhenian Sea.
Rome is the capital city of **Italy**. It is located in the central-western portion of the Italian Peninsula, along the Tiber River.



## Homework

Play with different data models on your favourite domain (e.g. stocks) providing relevant data. Enjoy tructured decoding magic.